In [162]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [163]:
from crewai import Agent, Task, Crew

In [164]:
import os

os.environ['OPENAI_API_KEY'] = 'Api-keys'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = 'api-keys'

## crewAI Tools

In [165]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [166]:
financial_data_agent

## Creating Agents

In [167]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [168]:
strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and checking various Earnings Conference Calls(56-1 for thai firm) based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines prediction based on the past 4 quater. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [169]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [170]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights "
         "associated with potential activities and news on firm to trader.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks and recent news. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to predict trends and "
              "news activities align with the firm’s risk tolerance. and take data to give inform about the next quarter",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [171]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data and Financial statement for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and past 4 Quarter of overall summary of earnings call transcript of selected stocks."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [172]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and predicting strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of predicted next quarter earning call for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=strategy_agent,
)


In [173]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved analyst strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute plan for {stock_selection}."
    ),
    agent=execution_agent,
)


In [174]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the predicted "
        "next Quarter Income and value driver in the same type of company and execution plans for {stock_selection}. "
        "Provide a predicting Earnings Conference Calls of next quarter  "
        "and suggest detailed analysis of potential key factor like revenue, operating margin, eps and comparing with past quarter to support evidence."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [175]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[
            data_analyst_agent, 
            strategy_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-08-25 22:25:24,788 - 8688 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [179]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'Kbank',
    'initial_capital': '10000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Next Quarterly Prediction',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input


In [180]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data and Financial statement for the selected stock (Kbank). Use statistical modeling and machine learning to identify trends and past 4 Quarter of overall summary of earnings call transcript of selected stocks.


> Entering new CrewAgentExecutor chain...
I need to gather the necessary insights and alerts about significant market opportunities or threats for Kbank by continuously monitoring and analyzing market data and financial statements, and using statistical modeling and machine learning. I also need to gather the past 4 Quarters of overall summary of earnings call transcript of selected stocks.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data and financial statements for Kbank", "context": "The data analyst needs to use statistical modeling and machine learning to identify trends in the market dat

- Display the final result as Markdown.

In [181]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Kbank:

Introduction:
KASIKORNBANK (Kbank) has demonstrated a strong financial performance in the first and second quarters of 2024. The bank reported a net profit of Baht 38,528 million in the first quarter, with a net interest margin ratio of 3.76%. However, in the second quarter, the net profit decreased by 6.18% to Baht 12,653 million. Kbank aims to achieve a double-digit return on equity by 2026 and targets a 5% loan growth in 2024 under the '3+1' strategy. The bank is also focusing on growing fee income through its wealth advisory business. 

Risk Analysis:
1. Market Risk: The fluctuating economic conditions and market volatility may impact Kbank's revenue streams and profitability. Diversifying revenue sources and closely monitoring market trends is crucial to mitigate market risk.

2. Credit Risk: With the planned 8% growth in the loan portfolio, Kbank needs to ensure effective risk management practices to mitigate potential credit risks. Monitoring loan portfolios and implementing proactive measures are essential to reduce non-performing loans by the predicted 3%.

3. Operational Risk: Implementing cost-cutting measures to achieve the predicted 5% increase in net profit may pose operational risks. Close monitoring of expenses and timely adjustments to strategies are necessary to mitigate operational risks.

4. Regulatory Risk: Changes in regulations and compliance requirements can impact Kbank's operations and financial performance. Staying updated with regulatory changes and ensuring compliance with laws is essential to mitigate regulatory risks.

5. Liquidity Risk: With the expansion of lending operations, Kbank needs to manage liquidity effectively to meet the increased demand for loans. Maintaining a balance between assets and liabilities is crucial to mitigate liquidity risk.

Mitigation Recommendations:
1. Conduct regular stress tests to assess the bank's resilience to market shocks.
2. Enhance credit risk assessment processes and monitor loan portfolios closely.
3. Establish clear operational guidelines and contingency plans to address operational risks.
4. Stay informed about regulatory changes and ensure compliance with all laws and regulations.
5. Maintain adequate liquidity reserves and diversify funding sources to mitigate liquidity risk.

Conclusion:
By implementing the above mitigation recommendations and closely monitoring key risk factors, Kbank can navigate potential risks effectively and achieve its predicted earnings targets. Managing risks proactively will position Kbank for sustainable growth and success in the next quarter and beyond.